In [5]:
import csv
import urllib

# 1. 处理数据
- 处理csv数据

In [51]:
def loadCsvFunc(filename):
    lines = csv.reader(open(filename, "r"))
    dataset = list(lines)
    for i in range(len(dataset)):
        dataset[i] = [float(x) for x in dataset[i]]
    return dataset

In [52]:
filename = 'pima-indians-diabetes.data.csv'
dataset = loadCsvFunc(filename)
print('Loaded data file {%s} with {%d} rows' %(filename, len(dataset)))


Loaded data file {pima-indians-diabetes.data.csv} with {768} rows


# 2. 分割数据集

In [53]:
import random
def splitDataset(dataset, splitRatio):
	trainSize = int(len(dataset) * splitRatio)
	trainSet = []
	copy = list(dataset)
	while len(trainSet) < trainSize:
		index = random.randrange(len(copy))
		trainSet.append(copy.pop(index))
	return [trainSet, copy]

In [54]:
splitRatio = 0.67
train, test = splitDataset(dataset, splitRatio)

# 3. 提取数据特征
- 朴素贝叶斯模型包含训练数据集中数据的特征，然后使用这个数据特征来做预测
- 我们将数据特征的获取划分为以下的子任务：
   - 按类别划分数据
   - 计算均值
   - 计算标准差
   - 提取数据集特征
   - 按类别提取属性特征

## 3.1 按类别划分数据


In [57]:
def separateByClass(dataset):
	separated = {}
	for i in range(len(dataset)):
		vector = dataset[i]
		if (vector[-1] not in separated):
			separated[vector[-1]] = []
		separated[vector[-1]].append(vector)
	return separated

In [58]:
separeted = separateByClass(dataset)

## 3.2 计算均值
- 我们需要计算在每个类中每个属性的均值。均值是数据的中点或者集中趋势，在计算概率时，我们用它作为高斯分布的中值。
- 我们也需要计算每个类中每个属性的标准差。标准差描述了数据散布的偏差，在计算概率时，我们用它来刻画高斯分布中，每个属性所期望的散布。
- 标准差是方差的平方根。方差是每个属性值与均值的离差平方的平均数。注意我们使用N-1的方法（译者注：参见无偏估计），也就是在在计算方差时，属性值的个数减1。

In [60]:
import math
def mean(numbers):
    # 每个类中每个类的均值
	return sum(numbers)/float(len(numbers))
 
def stdev(numbers):
    # 每个类中每个属性的标准差（N-1）方法
	avg = mean(numbers)
	variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
	return math.sqrt(variance)

## 3.3 按类别提取特征值
- 合并代码，我们首先将训练数据集按照类别进行划分，然后计算每个属性的摘要。

In [65]:
def summarize(dataset):
	summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
	del summaries[-1]
	return summaries

def summarizeByClass(dataset):
	separated = separateByClass(dataset)
	summaries = {}
	for classValue, instances in separated.items():
		summaries[classValue] = summarize(instances)
	return summaries

In [66]:
summary = summarizeByClass(dataset)

# 4. 预测
- 我们现在可以使用从训练数据中得到的摘要来做预测。做预测涉及到对于给定的数据样本，计算其归属于每个类的概率，然后选择具有最大概率的类作为预测结果。
- 我们可以将这部分划分成以下任务：
    - 计算高斯概率密度函数
    - 计算对应类的概率
    - 单一预测
    - 评估精度

## 4.1 计算高斯概率密度函数
- 给定来自训练数据中已知属性的均值和标准差，我们可以使用高斯函数来评估一个给定的属性值的概率。
- 在calculateProbability()函数中，我们首先计算指数部分，然后计算等式的主干。这样可以将其很好地组织成2行。


In [69]:
import math
def calculateProbability(x, mean, stdev):
	exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
	return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

## 4.2 计算所属类的概率
- 我们可以计算一个属性属于某个类的概率，那么合并一个数据样本中所有属性的概率，最后便得到整个数据样本属于某个类的概率
- 使用乘法合并概率,在下面的calculClassProbilities()函数中，给定一个数据样本，它所属每个类别的概率，可以通过将其属性概率相乘得到。结果是一个类值到概率的映射

In [71]:
def calculateClassProbabilities(summaries, inputVector):
	probabilities = {}
	for classValue, classSummaries in summaries.items():
		probabilities[classValue] = 1
		for i in range(len(classSummaries)):
			mean, stdev = classSummaries[i]
			x = inputVector[i]
			probabilities[classValue] *= calculateProbability(x, mean, stdev)
	return probabilities

# 5. 预测
## 5.1 单一预测
- 既然可以计算一个数据样本属于每个类的概率，那么我们可以找到最大的概率值，并返回关联的类。
- 下面的predict()函数可以完成以上任务。


In [73]:
def predict(summaries, inputVector):
	probabilities = calculateClassProbabilities(summaries, inputVector)
	bestLabel, bestProb = None, -1
	for classValue, probability in probabilities.items():
		if bestLabel is None or probability > bestProb:
			bestProb = probability
			bestLabel = classValue
	return bestLabel

## 5.2 多重预测
- 最后，通过对测试数据集中每个数据样本的预测，我们可以评估模型精度。getPredictions()函数可以实现这个功能，并返回每个测试样本的预测列表。

In [74]:
def getPredictions(summaries, testSet):
	predictions = []
	for i in range(len(testSet)):
		result = predict(summaries, testSet[i])
		predictions.append(result)
	return predictions

# 6. 计算精度
- 预测值和测试数据集中的类别值进行比较，可以计算得到一个介于0%~100%精确率作为分类的精确度。getAccuracy()函数可以计算出这个精确率

In [75]:
def getAccuracy(testSet, predictions):
	correct = 0
	for x in range(len(testSet)):
		if testSet[x][-1] == predictions[x]:
			correct += 1
	return (correct/float(len(testSet))) * 100.0

In [88]:
def main():
	filename = 'pima-indians-diabetes.data.csv'
	splitRatio = 0.67
	dataset = loadCsvFunc(filename)
	trainingSet, testSet = splitDataset(dataset, splitRatio)
	# prepare model
	summaries = summarizeByClass(trainingSet)
	# test model
	predictions = getPredictions(summaries, testSet)
	accuracy = getAccuracy(testSet, predictions)
	print(accuracy)

In [89]:
main()

74.40944881889764
